In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cv2
from scipy import signal
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
import json

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df.info()

Thankfully no Nulls in data.
Below heatmaps for Geospatial density

In [ ]:
fig = px.density_heatmap(train_df, x="longitude", y="latitude", marginal_x="histogram", marginal_y="histogram", width=700, height=700, nbinsx=100, nbinsy=100)
fig.show(filename='basic-line')

In [ ]:
fig = px.density_heatmap(test_df, x="longitude", y="latitude", marginal_x="histogram", marginal_y="histogram", width=700, height=700, nbinsx=100, nbinsy=100)
fig.show(filename='basic-line')

In [ ]:
fig = px.scatter_mapbox(train_df,
                        lat="latitude",
                        lon="longitude",
                        zoom=4,
                        color="label",
                        height=900,
                        width=1300)
fig.update_layout(mapbox_style='carto-positron')
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
sns.pairplot(train_df, y_vars='label')
sns.pairplot(test_df)

Checking if labels are correlated to lat/long

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 5), sharey=True)
n_labels = len(np.unique(train_df['label']))
for index, label in enumerate(np.unique(train_df['label'])):
    data_ = train_df[train_df['label']==label]
    title = 'Class ' + str(int(label))
    sns.histplot(data=data_, x="longitude", y="latitude", cbar=True, bins=50, ax=axes[index]).set(title=title)
plt.show()

Checking if train/test split is proper

In [ ]:
plt.scatter(train_df['longitude'], train_df['latitude'])
plt.scatter(test_df['longitude'], test_df['latitude'])
plt.show()

Equalizing histograms for y_CrCb and HSV color space

In [ ]:
def hisEqulColor(img):
    ycrcb=cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
    channels=cv2.split(ycrcb)
    hist = cv2.equalizeHist(channels[0])
    ycrcb = cv2.merge((hist,  channels[1], channels[2]))
    img = cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2BGR)
    return img

def hisEqulColorHSV(img):
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    channels=cv2.split(hsv)
    hist = cv2.equalizeHist(channels[2])
    hsv = cv2.merge((channels[0],  channels[1], hist))
    img = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    return img

Combining 2 Sobel filters

In [ ]:
def combined_filter(img, mask1, mask2):
    conv1 = np.zeros(img.shape)
    conv2 = np.zeros(img.shape)
    for i in range(3):
        conv1[:, :, i] = signal.convolve2d(img[:, :, i], mask1, mode='same', boundary='symm')
        conv2[:, :, i] = signal.convolve2d(img[:, :, i], mask2, mode='same', boundary='symm')
    return np.sqrt(conv1**2 + conv2**2)

In [ ]:
sep_classes = [train_df[train_df['label']==0], train_df[train_df['label']==1], train_df[train_df['label']==2]]

In [ ]:
s1 = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
s2 = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])

In [ ]:
def edge_rec(class_n):
    n_pics = 4
    paths = sep_classes[class_n]['example_path'].sample(n_pics).to_list()
    fig, ax = plt.subplots(n_pics, 5)
    fig.set_size_inches(15,12)
    for i, path in enumerate(paths):
        img_ = cv2.imread(path)
        img_ycrcb = hisEqulColor(img_)
        img_hsv = hisEqulColorHSV(img_)
        ax[i, 0].imshow(img_)
        ax[i, 1].imshow(img_ycrcb)
        ax[i, 2].imshow(img_hsv)
        ax[i, 3].imshow(combined_filter(img_ycrcb/255,s1,s2))
        ax[i, 4].imshow(combined_filter(img_hsv/255,s1,s2))
    plt.show()

Seeing how filters work on different classes

In [ ]:
edge_rec(0)
edge_rec(1)
edge_rec(2)

Preparing data for some simple naive models

In [ ]:
y = np.array(train_df['label']).reshape(-1,1)

In [ ]:
X = np.array(train_df[['latitude','longitude']])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42)

In [ ]:
tree_depth = 3
tree_clf = DecisionTreeClassifier(max_depth=tree_depth, random_state=42)
tree_clf.fit(x_train,y_train)

f1tr = f1_score(y_train,tree_clf.predict(x_train), average='weighted')
f1te = f1_score(y_test,tree_clf.predict(x_test), average='weighted')
acctr = accuracy_score(y_train,tree_clf.predict(x_train))
accte = accuracy_score(y_test,tree_clf.predict(x_test))

print(f1tr, f1te, acctr, accte)

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=10)
rnd_clf.fit(x_train,y_train)

f1tr = f1_score(y_train,rnd_clf.predict(x_train), average='weighted')
f1te = f1_score(y_test,rnd_clf.predict(x_test), average='weighted')
acctr = accuracy_score(y_train,rnd_clf.predict(x_train))
accte = accuracy_score(y_test,rnd_clf.predict(x_test))

print(f1tr, f1te, acctr, accte)

In [ ]:
gbc_clf = GradientBoostingClassifier(n_estimators=10)
gbc_clf.fit(x_train,y_train)

f1tr = f1_score(y_train,gbc_clf.predict(x_train), average='weighted')
f1te = f1_score(y_test,gbc_clf.predict(x_test), average='weighted')
acctr = accuracy_score(y_train,gbc_clf.predict(x_train))
accte = accuracy_score(y_test,gbc_clf.predict(x_test))

print(f1tr, f1te, acctr, accte)

Read in train pictures

In [ ]:
image_paths_per_class = train_df.groupby("label")["example_path"].apply(list).to_dict()
images_per_class = {
    label: [cv2.imread(path) for path in paths]
    for label, paths in image_paths_per_class.items()
}

Let's see the distribution of classes

In [ ]:
{label: len(images) for label, images in images_per_class.items()}

We will need to augment the data in order to make every class representative

Read in test pictures

In [ ]:
test_pictures_paths = test_df["example_path"].tolist()
test_images = [cv2.imread(path) for path in test_pictures_paths]

Converting data to proper formats

In [ ]:
labels = []
data = []
for label, pictures in images_per_class.items():
    labels.extend([label] * len(pictures))
    data.extend(pictures)

labels = tf.one_hot(labels, 3).numpy()
data = np.array(data)

Splitting the data into train and validation sets

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(data, labels, test_size=0.2)
print(len(y_valid[y_valid[:, 0] == 1]), len(y_valid[y_valid[:, 1] == 1]), len(y_valid[y_valid[:, 2] == 1]))

Let's scale the data

In [ ]:
X_train_unprocessed = X_train / 255
X_valid_unprocessed = X_valid / 255

Let's define metrics

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Let's train model on unprocessed data to have comparison

In [ ]:
model_unprocessed = keras.Sequential(
    [
        keras.layers.Input(shape=(332, 332, 3)),
        keras.layers.Conv2D(16, (3,3)),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Conv2D(16, (3,3)),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Conv2D(16, (5,5)),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Flatten(),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dense(64, activation="leaky_relu"),
        keras.layers.Dense(8, activation="leaky_relu"),
        keras.layers.Dense(3, activation="softmax")
    ]
)
model_unprocessed.summary()

In [ ]:
model_unprocessed.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy", recall_m, precision_m, f1_m])

In [ ]:
history = model_unprocessed.fit(X_train_unprocessed, y_train, validation_data=(X_valid_unprocessed, y_valid), epochs=15)

In [ ]:
def plot_model_performance(history):
    # extract loss and accuracy of model
    model_loss = history["loss"]
    model_val_loss = history["val_loss"]
    model_acc = history["accuracy"]
    model_val_acc = history["val_accuracy"]
    model_recall = history["recall_m"]
    model_val_recall = history["val_recall_m"]
    model_precission = history["precision_m"]
    model_val_precission = history["val_precision_m"]
    model_f1 = history["f1_m"]
    model_val_f1 = history["val_f1_m"]
    epochs = range(len(model_loss))

    # create subplot
    plt.clf()
    fig, axs = plt.subplots(5, 1, sharex=True)
    fig.set_size_inches(10, 10)

    # loss
    axs[0].plot(epochs, model_loss, 'b', label='Training loss')
    axs[0].plot(epochs, model_val_loss, 'r', label='Validation loss')
    axs[0].title.set_text('Training and validation loss')
    axs[0].set(ylabel="Loss")
    axs[0].legend()

    # accuracy
    axs[1].plot(epochs, model_acc, 'b', label='Training accuracy')
    axs[1].plot(epochs, model_val_acc, 'r', label='Validation accuracy')
    axs[1].title.set_text('Training and validation accuracy')
    axs[1].set(ylabel="Accuracy")
    axs[1].legend()
    axs[1].set_ylim([0, 1])

     # recall
    axs[2].plot(epochs, model_recall, 'b', label='Training recall')
    axs[2].plot(epochs, model_val_recall, 'r', label='Validation recall')
    axs[2].title.set_text('Training and validation recall')
    axs[2].set(ylabel="Recall")
    axs[2].legend()
    axs[2].set_ylim([0, 1])

    # precission
    axs[3].plot(epochs, model_precission, 'b', label='Training precision')
    axs[3].plot(epochs, model_val_precission, 'r', label='Validation precision')
    axs[3].title.set_text('Training and validation precision')
    axs[3].set(ylabel="Precision")
    axs[3].legend()
    axs[3].set_ylim([0, 1])

    # f1
    axs[4].plot(epochs, model_f1, 'b', label='Training f1 score')
    axs[4].plot(epochs, model_val_f1, 'r', label='Validation f1 score')
    axs[4].title.set_text('Training and validation f1 score')
    axs[4].set(ylabel="F1 score")
    axs[4].legend()
    axs[4].set_ylim([0, 1])

    fig.text(0.5, 0.04, 'Epochs', ha='center')
    plt.show()

In [ ]:
plot_model_performance(history.history)


Preprocess data using hsv and filters

In [ ]:
def preprocess_picture(img):
    img_hsv_normalized = hisEqulColorHSV(img)
    filtered_hsv = combined_filter(img_hsv_normalized/255,s1,s2)
    return filtered_hsv

In [ ]:
X_train_preprocessed = np.array([preprocess_picture(img) for img in X_train])
X_valid_preprocessed = np.array([preprocess_picture(img) for img in X_valid])

Creating new model that will be trained on preprocessed data

In [ ]:
model_processed = keras.Sequential(
    [
        keras.layers.Input(shape=(332, 332, 3)),
        keras.layers.Conv2D(16, (3,3)),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Conv2D(16, (3,3)),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Conv2D(16, (5,5)),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Flatten(),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dense(64, activation="leaky_relu"),
        keras.layers.Dense(8, activation="leaky_relu"),
        keras.layers.Dense(3, activation="softmax")
    ]
)
model_processed.summary()

In [ ]:
model_processed.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy", recall_m, precision_m, f1_m])

In [ ]:
history_processed = model_unprocessed.fit(X_train_preprocessed, y_train, validation_data=(X_valid_preprocessed, y_valid), epochs=15)

Plotting performance

In [ ]:
plot_model_performance(history_processed.history)

As we see the model did not perform better than the base one, let's try to increase size of the kernels, shrink down the linear layers size and add dropout as we have seen that model is overfitting

In [ ]:
model_processed_2 = keras.Sequential(
    [
        keras.layers.Input(shape=(332, 332, 3)),
        keras.layers.Conv2D(16, (5,5)),
        keras.layers.MaxPool2D(),
        keras.layers.Conv2D(16, (5, 5)),
        keras.layers.MaxPool2D(),
        keras.layers.Conv2D(16, (3,3)),
        keras.layers.MaxPool2D(),
        keras.layers.Flatten(),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64, activation="leaky_relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(8, activation="leaky_relu"),
        keras.layers.Dense(3, activation="softmax")
    ]
)
model_processed_2.summary()

As we have seen previously the data need to be augmented

In [ ]:
model_processed_2.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy", recall_m, precision_m, f1_m])

In [ ]:
history_processed_2 = model_processed_2.fit(X_train_preprocessed, y_train, validation_data=(X_valid_preprocessed, y_valid), epochs=20)

In [ ]:
plot_model_performance(history_processed_2.history)

It seems that bigger model did not solve our issue - it shows huge overfitting, let's try keeping the kernel size and let's shrink amount and size of Dense layers and change their activation layer

In [ ]:
model_processed_3 = keras.Sequential(
    [
        keras.layers.Input(shape=(332, 332, 3)),
        keras.layers.Conv2D(16, (5,5)),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Conv2D(12, (3, 3), activation="leaky_relu"),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Conv2D(8, (3,3), activation="leaky_relu"),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Flatten(),
        keras.layers.Dense(256, activation="leaky_relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(32, activation="tanh"),
        keras.layers.Dense(3, activation="softmax")
    ]
)
model_processed_3.summary()

In [ ]:
model_processed_3.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy", recall_m, precision_m, f1_m])

In [ ]:
history_processed_3 = model_processed_3.fit(X_train_preprocessed, y_train, validation_data=(X_valid_preprocessed, y_valid), epochs=15)

In [ ]:
plot_model_performance(history_processed_3.history)

Augmenting the data

In [ ]:
per_class_counts = {
    label: len(y_train[y_train[:, label] == 1])
    for label in range(3)
}
max_count = max(per_class_counts.values())
deficit = {label: abs(per_class_counts[label] - max_count) for label in per_class_counts}
added_labels, added_images = [], []
for label, class_deficit in deficit.items():
    pool_of_images = X_train_preprocessed[y_train[:,label] == 1]
    indices = np.random.choice(np.arange(len(pool_of_images)), deficit[label]).astype(np.int32)
    if len(indices):
        images_to_add = pool_of_images[indices]
        added_labels.extend([label] * class_deficit)
        added_images.append(images_to_add)

added_images = np.vstack(added_images)
added_labels = tf.one_hot(added_labels, 3)
X_train_preprocessed = np.vstack([X_train_preprocessed, added_images])
y_train = np.vstack([y_train, added_labels])

In [ ]:
model_augmented = keras.Sequential(
    [
        keras.layers.Input(shape=(332, 332, 3)),
        keras.layers.Conv2D(16, (5,5)),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Conv2D(10, (4, 4), activation="leaky_relu"),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Conv2D(6, (3,3), activation="leaky_relu"),
        keras.layers.MaxPool2D((3,3)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="leaky_relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(32, activation="tanh"),
        keras.layers.Dense(3, activation="softmax")
    ]
)
model_augmented.summary()

In [ ]:
model_augmented.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy", recall_m, precision_m, f1_m])

In [ ]:
history_augmented = model_augmented.fit(X_train_preprocessed, y_train, validation_data=(X_valid_preprocessed, y_valid), epochs=15)

In [ ]:
plot_model_performance(history_augmented.history)

In [ ]:
chosen_model = model_unprocessed
paths = test_df["example_path"].tolist()
ids = [path.split('/')[2][:-4] for path in paths]
test_images_np = np.array(test_images)
test_images_scaled = test_images_np / 255
result_dict = {"target": {}}
predictions = chosen_model.predict(test_images_scaled)
predictions_int = np.argmax(predictions, axis=1)
for idx, prediction in zip(ids, predictions_int):
    result_dict["target"][idx] = int(prediction)

with open("predictions.json", "w+") as fout:
    json.dump(result_dict, fout)